# Data Cleaning and Normalizing

## Notebook Setup

### Libraries

In [ ]:
## import modules
import numpy as np
import pandas as pd

### Notebook Settings

In [ ]:
## Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## disable the Pandas "setting a copy of a slice" warning
pd.options.mode.chained_assignment = None

## set default number of DataFrame rows printed to 8
pd.set_option('display.max_rows', 20)

### The Data

This notebook uses the following data set:

> `data > customers.csv`

The data is intended to track signups for credit cards.  The customer signs up under one or more promotional programs:  'bronze', 'silver' or 'gold'.  They are also perceived to be either from a low-population (Rural) or higher-population (Urban) area.  

In [ ]:
## import data
df_customer = pd.read_csv("data/customers.csv")
df_customer

## Working with Strings in Python

The <B>.str</B> attribute of the pandas <B>Series</B> gives us access to string methods and functionality when working with string data.  We'll find this extremely helpful in text-related situations.

First, let's look at how strings work with basic Python.  Some useful string methods are:  

- `.find("pattern")`: find the index of a pattern inside a string
- `.count("pattern")`: count the occurences of a pattern in the string
- `.upper()`: returns an uppercase copy of the string
- `.startswith("pattern")`: test if a pattern starts a string
- `.endswith("pattern")`: test if a pattern ends a string

In [ ]:
file1 = "january_data.csv"
file2 = "january_data.xlsx"

file1.startswith("jan")

In [ ]:
file2.endswith(".csv")

In [ ]:
file2.upper()

### "Parsing" (Spliting and Slicing) Strings

The two principal ways we like to "subdivide" strings are splitting and slicing:

In [ ]:
# splitting on a delimiter
line = 'I,have,happy,feet'
items = line.split(',')
items

# no argument: splitting on 'whitespace' 
# (includes any spaces, tabs, newlines)
line = 'I   have happy  feet.   '
items = line.split()
items

# slicing
date = '20250309'
year = date[0:4]
year

### Concatenate Strings

We can also concatenate strings using the `+` operator.

In [ ]:
"Here is the first file: " + file1

## Pandas String Methods
---
Pandas provides many very powerful tools for manipulating strings, based on the tools found in standard Python. 

### Vectorized Operations

One of the most important features of working with Pandas string columns is that operations are vectorized/broadcast.  Instead of looping through each item, we can just work with the entire column at once when using a Pandas DataFrame.

To access string methods, we first use the `.str` accessor. Most of the DataFrame/Series string methods have similar names as their base Python counterparts.

In [ ]:
# produce a new Series with FirstName uppercased
df_customer.FirstName.str.upper()                    # df_customer['FirstName']

You can also concatenate columns together.  Even the static string ", " is broadcast across items in the resulting Series.

In [ ]:
# produce a new Series with each corresponding value of 2 Series concatena
df_customer.LastName + ", " + df_customer.FirstName

### `.split()` method

As `split()` returns a list of values based on a delimiter, we can broadcast this across rows in a Series, resulting in a DataFrame with two values per row. 

If we wish to split on whitespace, we leave the position argument out.  This will see "one or more spaces" as the "split character".  

In the following example we use the `expand=True` argument to get all the elements separated into their own columns.

Note that absent values are noted with the `None` value.  This is not the pandas/numpy `NaN` value; it is Python's version of it.

In [ ]:
df_fixed_names = df_customer.FirstName.str.split(expand=True)
df_fixed_names

### setting a slice of a DataFrame

The None values in the middle initial column should be set to be proper missing values (numpy's `NaN` value).  Currently thanks to the split, they are `None`.  

First, we need to identify which rows have empty values.  

Then, we can use the below code inside of a call to `.loc[]`, but do all of that on the left side of the `=` assignment operator. On the right side of the operator we assign `np.nan` (the numpy/pandas null value, a.k.a. "Not a Number").

Unfortunately, comparisons to `None` do not work in pandas indexing tests, but... 
Fortunately the pandas `.isnull()` method will return `True` for `None` (as well as `NaN`) values.  

So what we are saying below is "set all of the empty values in the customer DataFrame identified by `.loc` column `1` to have the value `NaN`."

We'll discuss missing data, `NaN` and `.isnull()` more thoroughly below.

In [ ]:
# where column 1 is empty string, set to np.nan (Not a Number)
df_fixed_names.loc[  df_fixed_names[1].isnull(), 1  ] = np.nan
df_fixed_names

Let's now save these columns back into our main customer DataFrame.  Because the new Series are exactly the same length as the DataFrame from which they were generated, the values match when we assign them back.  (Note that `MiddleInitial` is the last column - we could use `.reindex()` to place it with the names if we desired.

Also note that I can use the attribute syntax to access the `FirstName` column, but to create a <I>new</I> column I must use the subscript.

In [ ]:
# note that the new MiddleInitial appears at the end of the table
df_customer.FirstName = df_fixed_names[0]
df_customer['MiddleInitial'] = df_fixed_names[1]
df_customer.head(3)

### `.extract()` method:  apply a pattern to isolate and extract parts of a string

We note that <B>SignUpProgram</B> has what appears to be a somewhat inconsistent set of values.  We can use <B>.unique()</B> to show all unique values in the Series, and <B>.value_counts()</B> to see how many of each:

In [ ]:
df_customer.SignUpProgram.unique()
print()

df_customer.SignUpProgram.value_counts()

We would prefer this column be consistent:  "bronze", "silver" or "gold".  Fortunately each of these values features that word somewhere, so we can just isolate it.  

We will use Pandas string methods to clean up this column. Specifically, we are going to do the following things:
1. convert all observations to lower case
2. extract the patterns ('gold', 'silver', 'bronze')
3. convert these patterns to title case

When extracting the pattern, we use the `|` operator to specify an `or`. What we are saying is "find the pattern *gold*, or the pattern *silver*, or the pattern *bronze*. Then, we use parentheses to say that we want to capture that pattern and extract it.

This is technically a type of regular expression, which is a powerful set of tools for specifying string patterns. For a thorough introduction to using regular expressions with Python see the following:

> ["Regular Expressions: Regexes in Python (Part 1)"](https://realpython.com/regex-python/) from the realpython.org website

In [ ]:
df_customer['SignUpProgramClean'] = (
    df_customer.SignUpProgram.str.lower()
                                .str.extract('(gold|silver|bronze)', expand=False)
                                .str.title()
) 
# expand=False ensures that only a Series, and not a DataFrame,
# will be returned (allowing us to continue with .str.title())

# (note I use parentheses to allow a statement to occupy multiple lines.
#  You could also use backslash, the 'line continuation character')

# use .value_counts() to see how many of each were extracted
df_customer.SignUpProgramClean.value_counts()

In [ ]:
# cross-tabulate the old value to the new to confirm our approach was correct
pd.crosstab(df_customer.SignUpProgramClean, df_customer.SignUpProgram)


For a more comprehensive treatment of string methods available in the Pandas module, see the following resources:

> ["Working with Strings"](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html) from the Python Data Science Handbook

> ["Working with Text Data"](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#working-with-text-data) from the Pandas User Guide

### `.replace()` method

This method works through a Series (or DataFrame) performing replacements, returning a new Series (or DataFrame). 

In [ ]:
df_customer['urban_rural_alt'] = (
    df_customer.UrbanRural.str.replace('Urban', 'City Folk')
)
df_customer.urban_rural_alt.value_counts()

We'll managing these "missing" values in the next section.

# Categorical Data

Any column that contains a limited number of unique values (`UrbanRural` with 'urban' and 'rural', or `SignUpProgramClean` with 'gold', 'silver' or 'bronze') contains categorical information (as contrasted with values like 'LastName' or 'Birthdate', which will have many different values).  

In [ ]:
# let's look at the data again:  keep in mind the new columns were added to the end
df_customer

df_customer.dtypes

The pandas dtype 'category' can be used to optimize our work with categorical values. 

The 'category' dtype specifies a fixed number of unique values, potentially with a specified order.  It should be noted too that because it is aware of valid types and internally uses an integer to represent each value (rather than repeated strings), the category type is more memory and time efficient, and will ensure the column's integrity (i.e., prevent unintended values from being set).  

In [ ]:
## 'object' dtype (default for string values)
df_customer.UrbanRural.dtype

You can cast such values to Categorical using `pd.Categorical()` :

In [ ]:
df_customer['UrbanRural_cat'] = pd.Categorical(df_customer.UrbanRural)

Now when we view the `.dtype` we see not only the new type, but we also see all unique values

In [ ]:
df_customer.UrbanRural_cat.dtype

### Harmonizing Levels

As often happens, it looks like there are some odd values, this time in the `UrbanRural` column, and in this case these odd values should be considered as missing or empty.  They may have been manually entered, or entered by different processes.  

We need to set the '*\*unknown\**', and the '*\*Missing\**' values to be missing.  With plain string type values we could do this overtly using code like the following, with `.loc[]` and the `.isin()` method:

In [ ]:
# Make new column
df_customer['UrbanRural2'] = df_customer.UrbanRural

# use .loc[] with the .isin() test to identify these values; then 
df_customer.loc[   df_customer.UrbanRural2.isin(['*unknown*','*Missing*']), 'UrbanRural2'   ] = np.NaN

# Check if fixed
df_customer.UrbanRural2.value_counts(dropna=False)

# Remove that column
del df_customer['UrbanRural2']

<B>Or</B>, you could do this harmonization implicitly (and much more conveniently!), by identifying valid category levels, and then implicitly setting other levels (anything not listed) to missing (`Nan`).

In [ ]:
# Change non-valid levels to missing 
df_customer['UrbanRural_cat'] = pd.Categorical(df_customer.UrbanRural,
                                               categories=['Urban', 'Rural'])

# Verify it is fixed
df_customer.UrbanRural_cat.value_counts(dropna=False, sort=False)
df_customer

### Renaming Categories

One more thing we may want to do is rename some of the categories.

The following `.cat.rename_categories()` method takes a dictionary as its argument, where we specify the old-to-new mapping.

In [ ]:
df_customer['UrbanRural_renamed'] = ( df_customer.UrbanRural_cat
                                      .cat.rename_categories({'Urban':'City', 
                                                              'Rural':'Outlying Areas'})
                                    )
df_customer.loc[:, ['UrbanRural_renamed', 'UrbanRural_cat']]

# Missing Data

Accounting for missing data is an important part of any analysis. We first need to be able to notice and quantify the missing data in our DataFrame.  We may wish to consider how the data came to be missing.  Finally we'll want to decide what to do with rows or columns that have missing data:  fill the missing data with a appropriate values, or drop the rows or columns entirely.  


Missing data mechanics: 
* Missing data is represented as NaN in a dataframe. 
* Pandas has dedicated methods for dealing with missing data appropriately. 


What kinds of things can we do when we have missing data?
<ul>
  <li>Quantify the missingness. How much is missing, and where?</li>
  <li>Flag the missing values so we can hopefully retrieve them by asking the source of our data to send us the missing elements. </li>
  <li>Exclude the rows with missing elements from our analysis</li>
  <li><I>Impute</I> the missing data, like substituting the mean of a numeric variable in its place, or forward-filling it with the last observed value.</li>
</ul>    
Concerning that last option, missing data imputation should not be taken lightly as it may distort the character or meaning of the existing data. 


### Boolean summary functions `any()` and `all()`
We will need these functions to summarize the missing values that we see.

`any()` will return `True` if <B><I>any</I></B> values in a container (or iterable) can be seen as `True` values.  
`all()` will return `True` if <B><I>all</I></B> values in a container (or iterable) can be seen as `True` values.

In [ ]:
print(f'any([NaN, False, False]): {any([0, 0, 0])}')
print()
print(f'any([False, False, True]): {any([0, 0, 1])}')
print()
print(f'all([True, True, NaN]): {all([1, 1, 0])}')
print()
print(f'all([True, True, True]): {all([1, 1, 1])}')

Besides `False` and `0`, also note that `NaN` or `None` values count as `False` as well.  

### Using `sum()` with `True` and `False` values

Of course `sum()` will sum up any numbers in its container (or iterable).  But when used with boolean values, `sum()` <I>counts the number of `True` values found</I>.  In other words when used with `sum()`, `True` is counted as `1`, and `False` is counted as `0`.  

In [ ]:
sum([True, True, False])
sum([False, False, False])

### Quantify Missing Data

We will start with detecting missing data. We can detect missing data using the `isnull()` method. You can then apply the `any()`, `all()` and `sum()` methods to summarize "missingness" by column or row.  You can also use the `any()` and `all()` functions on series or DataFrames to similar (but sometimes different) effect.

In [ ]:
# .isnull() on a DataFrame returns a matching DataFrame of booleans
df_customer.isnull()

In [ ]:
# are there any False values in the above DataFrame?
any(df_customer.isnull())              # True

In [ ]:
# which *columns* have missing data?
df_customer.isnull().any()

In [ ]:
# which *rows* have missing data?
df_customer.isnull().any(axis=1)

In [ ]:
# .isnull() on a Series returns a matching Series of booleans
df_customer.CustomerID.isnull()

# are there any False values in the CustomerID column?
any(df_customer.CustomerID.isnull())   # False

# same question (and answer) using the .any() method
df_customer.CustomerID.isnull().any()

In [ ]:
# How many missing per column?
pd.options.display.max_rows = 20

df_customer.isnull().sum()

pd.options.display.max_rows = 8

In [ ]:
# another easy way to get the count of non-nulls
df_customer.info()

### Dropping Missing Data:  `.dropna()`

Missing data can of course distort our view of the data.  We can drop any rows with missing data using `.dropna()`.

In [ ]:
# Check out the df first - note early rows have missing data
df_customer

In [ ]:
# Drop rows with missing data
dfc = df_customer.dropna()
dfc

Of course we may want to scrutinize only certain columns for missing data - if an essential column has a value that must be there, but the others could have missing data, then we can specify a subset of columns to check:

In [ ]:
# drop rows with data missing in only selected columns
df_customer.dropna(subset=['CustomerID', 'LastName'])

In [ ]:
# drop columns with missing data
# maybe less useful...
df_customer.dropna(axis=1).head(3)

### Replacing Missing Data:  `.fillna()`

We also have the choice of replacing any missing data, for example to fill missing values with a default value, such as empty string.

In [ ]:
df_customer.MiddleInitial.fillna('')

Or more commonly, we may wish to replace the missing values with an <I>imputed</I> value, such as the mean of all the values.  (Categorical values will correctly remain missing because there is no mean.)

In [ ]:
col_means = df_customer.mean(numeric_only=True)
col_means

df_customer_no_na = df_customer.fillna(col_means)
df_customer_no_na

Of course values like `CustomerID` and `Zipcode` don't have a meaningful mean, so we may wish to be selective:

In [ ]:
# make a copy to play with
df_customer_no_na2 = df_customer.copy()

# fill in missing values in CreditScore with mean (rounded to integer)
df_customer_no_na2.CreditScore.fillna(round(df_customer.CreditScore.mean()))

In [ ]:
# how many missing per column?
pd.options.display.max_rows = 100

df_customer_no_na.isnull().sum()

pd.options.display.max_rows = 8

We could also impute these values with the last value carried forward, or other options (Check the help for `ffill()` and `bfill()`).

In [ ]:
## Fill with last value carried forward
df_customer_no_na = df_customer.ffill()
df_customer_no_na

For explore this topic further, consider the following resources:

> ["Working with Missing Data"](https://pandas.pydata.org/docs/user_guide/missing_data.html#working-with-missing-data) tutorial from the Pandas User Guide

> ["Handling Missing Data"](https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html) from the Python for Data Science Handbook

# Dealing with Duplicates

Another common task when cleaning up your data is to identify duplicated observations. There are a couple of scenarios we will mention here where duplicates may occur:
- You can get duplicated rows from a bad merge earlier in your cleaning process. In this case finding the duplicates clues you in to the bad merge so you can go back and fix it.
- You may have received data that has duplicates, in which case you may want to just remove the duplicates.

In the simplest case where entire rows have ben duplicated, we can use some basic Pandas methods. We will use the `duplicated()` method to find duplicated rows. By default, the first duplicated row is not considered a duplicate, with any following rows considered duplicates.

We also use the Pandas `isin()` method below to find IDs that are in a Series of IDs that we identified as duplicated.

In [ ]:
df_customer.duplicated()

In [ ]:
# are there any duplicated rows (any True values)?
df_customer.duplicated().any()

In [ ]:
# how many duplicated rows?
df_customer.duplicated().sum()

In [ ]:
# look at the duplicated rows
dup_ids = df_customer.loc[df_customer.duplicated()].CustomerID
dup_ids

list(dup_ids)

In [ ]:
# there were 9 rows with duplicated ids found, so the total of these ids is 18
df_customer.CustomerID.isin(dup_ids).sum()

In [ ]:
df_customer.duplicated().sum()

In [ ]:
df_customer.loc[df_customer.CustomerID.isin(dup_ids), :]             \
                                      .sort_values(['CustomerID'])   \
                                      .head(4)

In [ ]:
df_customer.shape
df_customer.drop_duplicates(inplace=True)
df_customer.shape

You may want to also consider whether just a subset of your columns have been duplicated, potentially hiding more subtle issues with your data. We use the `subset=` argument to check for duplicates across just specific columns.

In [ ]:
df_customer.columns

In [ ]:
# lets just check IDs and names
id_cols = ['CustomerID','FirstName','LastName']

# are there any duplicated rows?
df_customer.duplicated(subset=id_cols).any()

In [ ]:
# how many duplicated rows?
df_customer.duplicated(subset=id_cols).sum()

In [ ]:
# look at the duplicated rows
dup_ids = df_customer.loc[df_customer.duplicated(subset=id_cols), 'CustomerID']

df_customer.loc[df_customer.CustomerID.isin(dup_ids), :].sort_values(['CustomerID'])
# got one!

Can you see the issue that caused the duplication above? 

In [ ]:
df_customer.drop_duplicates(inplace=True, subset=id_cols)

# is there the same number of rows as unique IDs now?
df_customer.shape
df_customer.CustomerID.nunique()

In [ ]:
# save the customer data
df_customer.to_csv('data/customers_clean_temp.csv', index=False)